In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import datetime 
import time

In [4]:
#from matplotlib import pyplot as plt

In [24]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.impute import SimpleImputer

In [12]:
#!git
# !git commit -m "sprawdzam czy da sie zrobic commita w notebooku"
# 
# update: da się!

# 0. pozyskanie danych

In [4]:
movies =[]
url=['https://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=3&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=4&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=5&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=6&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=7&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=8&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=9&view=releasedate&view2=domestic&yr=2018&p=.htm'
    ]

start = time.time()

for i in range(len(url)): 
    time.sleep(1) #zeby nie dostac bana
    page = requests.get(url[i])
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    tr_elements = tr_elements[9:109]
    mov = []
    for xd in range(len(tr_elements)):
        for a in tr_elements[xd]:      
            try:
                mov.append(int(a.text_content()))
            except:
                mov.append(a.text_content())
                pass
            try:
                url_mov = a.getchildren()[0].getchildren()[0].getchildren()[0].attrib['href'] #XD
                time.sleep(1) # znowu zeby nie dostac bana
                page2 = requests.get('https://www.boxofficemojo.com'+url_mov)
                doc2 = lh.fromstring(page2.content)
                tr_elements2 = doc2.xpath('//tr')
                mov.append(tr_elements2[7].getchildren()[0].getchildren()[0].text_content()) #distributor full name
                mov.append(tr_elements2[7].getchildren()[1].getchildren()[0].text_content()) #release date
                mov.append(tr_elements2[8].getchildren()[0].getchildren()[0].text_content()) # genre
                mov.append(tr_elements2[8].getchildren()[1].getchildren()[0].text_content()) #runtime
                mov.append(tr_elements2[9].getchildren()[0].getchildren()[0].text_content()) #rating
                mov.append(tr_elements2[9].getchildren()[1].getchildren()[0].text_content()) #budget
                if(tr_elements2[15].getchildren()[0].text_content().replace(u'\xa0', u'').replace('+','') == 'Foreign:'):
                    mov.append(tr_elements2[15].getchildren()[1].text_content().replace(u'\xa0', u'')) #rest world
                else:
                    mov.append('no dist')
            except: 
                pass #XD2
        movies.append(mov)
        mov = []
        
end = time.time()
print(str((end - start)/60)+" minut") # 26 minut

25.812677562236786 minut


In [5]:
headers =['#','name','distributor_full_name','release_date','genre','runtime','rating','budget','rest_world','studio','total_gross','theaters_total','opening','theaters_opening','open_date','close_date',]
df = pd.DataFrame(movies,columns = headers)
df.to_csv('movies.csv')
# recznie usnalem wszystko po 878 - > movies3.csv

# 1. czyszczenie + imdb, cinemascore

In [41]:
df = pd.read_csv('movies.csv')
df.head()

,Unnamed: 0,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,total_gross,theaters_total,opening,theaters_opening,open_date,close_date
0,0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,2 hrs. 20 min.,PG-13,NaN,"$646,853,595",BV,"$700,059,566","4,084","$202,003,951","4,020",2/16,8/9
1,1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,2 hrs. 36 min.,PG-13,NaN,"$1,369,544,272",BV,"$678,815,482","4,474","$257,698,183","4,474",4/27,9/13
2,2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,1 hrs. 58 min.,PG,NaN,"$634,223,615",BV,"$608,581,744","4,410","$182,687,905","4,410",6/15,12/13
3,3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,2 hrs. 8 min.,PG-13,$170 million,"$891,764,701",Uni.,"$417,719,760","4,485","$148,024,610","4,475",6/22,10/4
4,4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,2 hrs. 19 min.,PG-13,NaN,"$812,700,000",WB,"$335,061,807","4,184","$67,873,522","4,125",12/21,4/4


In [1]:
#df['total_gross'].isnull().values.any()
# #usuwanie $ itp
# df['total_gross'].apply(lambda x: str(x))
# df['opening'].apply(lambda x: str(x))
# df['total_gross'] = df['total_gross'].replace('\.','', regex=True).replace(',','', regex=True).str[1:].astype(float)
# df['opening'] = df['opening'].replace('\.','', regex=True).replace(',','', regex=True).str[1:].astype(float)
# #df['opening'].fillna(df['opening'].mean(),inplace=True)
# df = df.sample(frac=1)
# # 'tasowanie' wierszy
#df.to_csv('movies.csv')
# df.head()

In [31]:
# aa=[]
# for a in range(100): aa.append(df['open_date'][a].split("-")[0])
# set(aa)

{'1',
 '10',
 '11',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'Apr',
 'Aug',
 'Dec',
 'Feb',
 'Jan',
 'Jul',
 'Jun',
 'Mar',
 'May',
 'Nov',
 'Oct',
 'Sep'}

In [39]:
def str2int_w_na(col):
    df[col] = df[col].fillna(-1)
    df[col] = df[col].astype(int)
    df[col] = df[col].astype(str)
    df[col] = df[col].replace('-1', np.nan)
    
def zamaianaDI(x):
    try:
        int(x)
        return x
    except:
        return {
            'Apr':4,
             'Aug':8,
             'Dec':12,
             'Feb':2,
             'Jan':1,
             'Jul':7,
             'Jun':6,
             'Mar':3,
             'May':5,
             'Nov':11,
             'Oct':10,
             'Sep':9
        }[x]
zamaianaDI('Apr')

4

In [50]:
df = pd.read_csv('movies.csv')
df = df.drop(['Unnamed: 0'], axis=1)

# len(df['studio'].unique()) #57
# len(df['genre'].unique()) #34
# len(df['rating'].unique()) #5

#runtime to int
df = df.dropna(subset=['runtime'])
df['runtime'] = df['runtime'].replace('hrs.','', regex=True).replace('min.','', regex=True).apply(lambda x: int(x.split()[0])*60 + int(x.split()[1]))

#budget to float (in milions)
df['budget'] = df['budget'].replace(' ','', regex=True).replace('million','', regex=True).str[1:]
df.loc[132,'budget'] =1
df['budget'] = df['budget'].astype(float)

#theters to int
df['theaters_total'] = df['theaters_total'].replace(',','', regex=True)
df['theaters_opening'] = df['theaters_total'].replace(',','', regex=True)
str2int_w_na('theaters_total')
str2int_w_na('theaters_opening')

#rest_wrorld to int
df['rest_world'] = df['rest_world'].replace('no dist',float('nan'), regex=True)
df['rest_world'] = df['rest_world'].replace('\.','', regex=True).replace(',','', regex=True).str[1:]
str2int_w_na('rest_world')

#total_gross, opening to int
df['total_gross'] = df['total_gross'].replace('\.','', regex=True).replace(',','', regex=True).str[1:]
df['total_gross'] = df['total_gross'].astype(int)

df['opening'] = df['opening'].replace('\.','', regex=True).replace(',','', regex=True).str[1:]
str2int_w_na('opening')

#days in theaters
for x in range(len(df)):
    try:
        d1 = datetime.date(2018, int(df['open_date'][x].split("/")[0]), int(df['open_date'][x].split("/")[1]) )
        d2 = datetime.date(2018, int(df['close_date'][x].split("/")[0]), int(df['close_date'][x].split("/")[1]) )
        days = d2 - d1
        if days<datetime.timedelta(0):
            df.loc[x,'days_in_theaters']= (datetime.timedelta(365) + (d2 - d1)).days
        else:
            df.loc[x,'days_in_theaters']= (d2 - d1).days
    except:
        pass #przepraszam ;_;

df.head()


,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,total_gross,theaters_total,opening,theaters_opening,open_date,close_date,days_in_theaters
0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140,PG-13,NaN,646853595,BV,700059566,4084,202003951,4084,2/16,8/9,174.0
1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156,PG-13,NaN,1369544272,BV,678815482,4474,257698183,4474,4/27,9/13,139.0
2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118,PG,NaN,634223615,BV,608581744,4410,182687905,4410,6/15,12/13,181.0
3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128,PG-13,170.0,891764701,Uni.,417719760,4485,148024610,4485,6/22,10/4,104.0
4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139,PG-13,NaN,812700000,WB,335061807,4184,67873522,4184,12/21,4/4,104.0


In [90]:
(datetime.timedelta(365) + (d2 - d1)).days

86

In [51]:
#imdb oceny
from imdb import IMDb

ia = IMDb()

In [52]:
import time

start = time.time()

for id in range(len(df)):
    try:
        movies = ia.search_movie(df.loc[id,'name'])
        movie = ia.get_movie(movies[0].movieID)
        df.loc[id,'imdb_rating']= movie['rating']
        directors=[]
        for person in movie['director']:    
            directors.append(person['name'])
        df.loc[id,'directors'] = ' & '.join(directors)
    except:
        df.loc[id,'directors'] = '???'
        
end = time.time()
print(end - start)
# 14 minut !!!
# 47 minut
df.head()

2844.142557621002


,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,total_gross,theaters_total,opening,theaters_opening,open_date,close_date,days_in_theaters,imdb_rating,directors
0,1.0,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140.0,PG-13,NaN,646853595,BV,700059566.0,4084,202003951,4084,2/16,8/9,174.0,7.3,Ryan Coogler
1,2.0,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156.0,PG-13,NaN,1369544272,BV,678815482.0,4474,257698183,4474,4/27,9/13,139.0,8.5,Anthony Russo & Joe Russo
2,3.0,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118.0,PG,NaN,634223615,BV,608581744.0,4410,182687905,4410,6/15,12/13,181.0,7.7,Brad Bird
3,4.0,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128.0,PG-13,170.0,891764701,Uni.,417719760.0,4485,148024610,4485,6/22,10/4,104.0,6.2,J.A. Bayona
4,5.0,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139.0,PG-13,NaN,812700000,WB,335061807.0,4184,67873522,4184,12/21,4/4,104.0,7.1,James Wan


In [ ]:
#df[['name','directors','imdb_rating']]
#len(df['directors'].unique()) #190

In [53]:
#cinemascore
import cinemascore

start = time.time()

for id in range(len(df)):
    try:
        x = cinemascore.search(df.loc[id,'name'])
        df.loc[id,'cinemascore_name'] = list(x)[0]
        df.loc[id,'cinemascore_rating'] = x[list(x)[0]]
    except:
        df.loc[id,'cinemascore_rating'] = np.nan
        pass
    
end = time.time()
print(str((end - start)/60)+" minut") # aprox. 5 min
# 26 min

25.963711607456208 minut


In [54]:
#pip install rotten_tomatoes_client
from rotten_tomatoes_client import RottenTomatoesClient

In [55]:
start = time.time()

for id in range(len(df)):
    try:
        result = RottenTomatoesClient.search(term=df.loc[id,'name'], limit=5)

        for i in range(len(result['movies'])):
            if(df.loc[id,'name'] == result['movies'][i]['name']) and (result['movies'][i]['year']==2018):
                df.loc[id,'RT_name'] = result['movies'][i]['name']
                df.loc[id,'RT_rating'] = result['movies'][i]['meterScore']
                break
    except:
        df.loc[id,'RT_name'] = np.nan
        df.loc[id,'RT_rating'] = np.nan
        pass
    
end = time.time()
print(str((end - start)/60)+" minut") # aprox. 5 min
#14 min


13.82043108145396 minut


In [68]:
# result = RottenTomatoesClient.search(term=df.loc[0,'name'], limit=5)
# result['movies'][3]['year']

2018

In [45]:
# result = RottenTomatoesClient.search('black panther', limit=5)
# range(len(result['movies']))

In [56]:
df.head(30)

,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,...,theaters_opening,open_date,close_date,days_in_theaters,imdb_rating,directors,cinemascore_name,cinemascore_rating,RT_name,RT_rating
0,1.0,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140.0,PG-13,NaN,646853595,BV,...,4084,2/16,8/9,174.0,7.3,Ryan Coogler,BLACK PANTHER (2018),A+,Black Panther,97.0
1,2.0,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156.0,PG-13,NaN,1369544272,BV,...,4474,4/27,9/13,139.0,8.5,Anthony Russo & Joe Russo,AVENGERS: INFINITY WAR (2018),A,Avengers: Infinity War,85.0
2,3.0,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118.0,PG,NaN,634223615,BV,...,4410,6/15,12/13,181.0,7.7,Brad Bird,"INCREDIBLES 2, THE (2018)",A+,Incredibles 2,94.0
3,4.0,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128.0,PG-13,170.0,891764701,Uni.,...,4485,6/22,10/4,104.0,6.2,J.A. Bayona,JURASSIC WORLD: FALLEN KINGDOM (2018),A-,Jurassic World: Fallen Kingdom,48.0
4,5.0,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139.0,PG-13,NaN,812700000,WB,...,4184,12/21,4/4,104.0,7.1,James Wan,AQUAMAN (2018),A-,Aquaman,65.0
5,6.0,Deadpool 2,Fox,"May 18, 2018",Action,119.0,R,110.0,460455185,Fox,...,4349,5/18,10/18,153.0,7.8,David Leitch,DEADPOOL 2 (2018),A,Deadpool 2,83.0
6,7.0,Dr. Seuss' The Grinch (2018),Universal,"November 9, 2018",Animation,90.0,PG,75.0,240973508,Uni.,...,4141,11/9,2/14,97.0,6.3,Yarrow Cheney & Scott Mosier,NaN,NaN,NaN,NaN
7,8.0,Mission: Impossible - Fallout,Paramount,"July 27, 2018",Action,147.0,PG-13,178.0,570956000,Par.,...,4395,7/27,10/18,83.0,7.8,Christopher McQuarrie,MISSION: IMPOSSIBLE - FALLOUT (2018),A,Mission: Impossible - Fallout,97.0
8,9.0,Ant-Man and the Wasp,Buena Vista,"July 6, 2018",Action,118.0,PG-13,NaN,406025399,BV,...,4206,7/6,11/1,118.0,7.1,Peyton Reed,ANT-MAN AND THE WASP (2018),A-,Ant-Man and the Wasp,88.0
9,10.0,Bohemian Rhapsody,Fox,"November 2, 2018",Drama,134.0,PG-13,52.0,687227217,Fox,...,4000,11/2,4/25,174.0,8.1,Bryan Singer,BOHEMIAN RHAPSODY (2018),A,Bohemian Rhapsody,61.0


In [22]:
df[['name','RT_name','RT_rating']].tail()

,name,RT_name,RT_rating
195,Mary and the Witch's Flower,NaN,NaN
196,Hearts Beat Loud,Hearts Beat Loud,91.0
197,1991,NaN,NaN
198,Papillon (2018),NaN,NaN
199,At Eternity's Gate,At Eternity's Gate,80.0


In [57]:
df.to_csv('movies_clean.csv',index=False)

In [ ]:
#df['budget'].unique()
#df['budget'].astype(float)
#df[df['budget']=='$170 million']
#df['budget'].isnull().sum() #104
#f[df['budget'].notnull()]
#df['budget'] = df['budget'].replace(' ','', regex=True).replace('million','', regex=True).str[1:]
#type(df.loc[36,'rest_world'])
#type(df['total_gross'][0])
#type(df['rest_world'][0])

In [ ]:
# critics reviews

# -key actors
# -director
# -season of year
# -other movies in cinema at that time
# -„hype index”
# china 

In [15]:
df = pd.read_csv('movies_clean.csv')
df.head()

,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,...,is_Documentary,is_Romance,studio_uni,studio_WB,studio_Fox,studio_Sony,studio_STX,studio_LGF,studio_BV,studio_Focus
0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140,PG-13,NaN,6.468536e+08,BV,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156,PG-13,NaN,1.369544e+09,BV,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118,PG,NaN,6.342236e+08,BV,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128,PG-13,170.0,8.917647e+08,Uni.,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139,PG-13,NaN,8.127000e+08,WB,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#df.groupby('genre').nunique().sort_values('#',ascending =bool(0))

In [64]:
for i in range(len(df)):
    try:
        if(df.loc[i,'genre'].find('Action') != -1):
            df.loc[i,'is_Action'] = 1
        else:
            df.loc[i,'is_Action'] = 0

        if(df.loc[i,'genre'].find('Animation') != -1):
            df.loc[i,'is_Animation'] = 1
        else:
            df.loc[i,'is_Animation'] = 0

        if(df.loc[i,'genre'].find('Drama') != -1):
            df.loc[i,'is_Drama'] = 1
        else:
            df.loc[i,'is_Drama'] = 0   

        if(df.loc[i,'genre'].find('Horror') != -1):
            df.loc[i,'is_Horror'] = 1
        else:
            df.loc[i,'is_Horror'] = 0

        if(df.loc[i,'genre'].find('Comedy') != -1):
            df.loc[i,'is_Comedy'] = 1
        else:
            df.loc[i,'is_Comedy'] = 0

        if(df.loc[i,'genre'].find('Adventure') != -1):
            df.loc[i,'is_Adventure'] = 1
        else:
            df.loc[i,'is_Adventure'] = 0

        if(df.loc[i,'genre'].find('Music') != -1):
            df.loc[i,'is_Music'] = 1
        else:
            df.loc[i,'is_Music'] = 0

        if(df.loc[i,'genre'].find('Sci-Fi') != -1):
            df.loc[i,'is_Sci-Fi'] = 1
        else:
            df.loc[i,'is_Sci-Fi'] = 0

        if(df.loc[i,'genre'].find('Comedy') != -1):
            df.loc[i,'is_Comedy'] = 1
        else:
            df.loc[i,'is_Comedy'] = 0

        if(df.loc[i,'genre'].find('Thriller') != -1):
            df.loc[i,'is_Thriller'] = 1
        else:
            df.loc[i,'is_Thriller'] = 0

        if(df.loc[i,'genre'].find('Documentary') != -1):
            df.loc[i,'is_Documentary'] = 1
        else:
            df.loc[i,'is_Documentary'] = 0

        if(df.loc[i,'genre'].find('Roman') != -1):
            df.loc[i,'is_Romance'] = 1    
        else:
            df.loc[i,'is_Romance'] = 0
        if(df.loc[i,'genre'].find('Fantasy') != -1):
            df.loc[i,'is_Fantasy'] = 1    
        else:
            df.loc[i,'is_Fantasy'] = 0
    except:
        pass


In [8]:
df[['name','genre', 'is_Action', 'is_Animation',
       'is_Drama', 'is_Music', 'is_Horror', 'is_Romance', 'is_Sci-Fi',
       'is_Thriller', 'is_Comedy', 'is_Adventure', 'is_Documentary']]
df.columns

Index(['#', 'name', 'distributor_full_name', 'release_date', 'genre',
       'runtime', 'rating', 'budget', 'rest_world', 'studio', 'total_gross',
       'theaters_total', 'opening', 'theaters_opening', 'open_date',
       'close_date', 'days_in_theaters', 'imdb_rating', 'directors',
       'cinemascore_rating', 'cinemascore_name', 'RT_name', 'RT_rating',
       'is_Action', 'is_Animation', 'is_Drama', 'is_Horror', 'is_Comedy',
       'is_Adventure', 'is_Music', 'is_Sci-Fi', 'is_Thriller',
       'is_Documentary', 'is_Romance'],
      dtype='object')

In [66]:
df.groupby('studio').nunique().sort_values('#',ascending =bool(0))

,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,...,is_Drama,is_Horror,is_Comedy,is_Adventure,is_Music,is_Sci-Fi,is_Thriller,is_Documentary,is_Romance,is_Fantasy
studio,,,,,,,,,,,,,,,,,,,,,
Fathom,146,146,1,134,25,72,6,0,0,1,...,2,2,2,2,2,2,2,2,2,2
WB,33,33,1,24,17,26,4,9,15,1,...,2,2,2,2,2,2,2,2,2,2
Magn.,27,27,1,25,9,18,5,0,1,1,...,2,2,2,1,1,1,2,2,1,1
IFC,26,26,1,26,7,20,4,0,3,1,...,2,2,2,1,1,1,2,2,1,1
WGUSA,22,22,1,19,14,21,2,0,4,1,...,2,2,2,2,1,2,2,1,1,2
Uni.,21,21,1,21,14,17,3,17,19,1,...,2,2,2,2,2,2,2,1,2,2
KL,18,18,1,17,8,17,2,0,0,1,...,2,2,2,2,1,1,2,2,2,1
Indic.,16,16,1,16,13,10,2,0,0,1,...,2,2,2,2,1,2,2,1,1,1
SPC,15,15,1,15,3,13,3,0,1,1,...,2,1,2,1,1,1,1,2,1,1


In [67]:
# studio
df.groupby('studio').nunique().sort_values('#',ascending =bool(0))

for i in range(len(df)):
    try:
        if(df.loc[i,'studio'].find('Uni.') != -1):
            df.loc[i,'studio_uni'] = 1
        else:
            df.loc[i,'studio_uni'] = 0

        if(df.loc[i,'studio'].find('WB') != -1):
            df.loc[i,'studio_WB'] = 1
        else:
            df.loc[i,'studio_WB'] = 0

        if(df.loc[i,'studio'].find('Fox') != -1):
            df.loc[i,'studio_Fox'] = 1
        else:
            df.loc[i,'studio_Fox'] = 0

        if(df.loc[i,'studio'].find('Sony') != -1):
            df.loc[i,'studio_Sony'] = 1
        else:
            df.loc[i,'studio_Sony'] = 0

        if(df.loc[i,'studio'].find('STX') != -1):
            df.loc[i,'studio_STX'] = 1
        else:
            df.loc[i,'studio_STX'] = 0

        if(df.loc[i,'studio'].find('LGF') != -1):
            df.loc[i,'studio_LGF'] = 1
        else:
            df.loc[i,'studio_LGF'] = 0

        if(df.loc[i,'studio'].find('BV') != -1):
            df.loc[i,'studio_BV'] = 1
        else:
            df.loc[i,'studio_BV'] = 0

        if(df.loc[i,'studio'].find('Focus') != -1):
            df.loc[i,'studio_Focus'] = 1
        else:
            df.loc[i,'studio_Focus'] = 0
            
        if(df.loc[i,'studio'].find('Fathom') != -1):
            df.loc[i,'studio_Fathom'] = 1
        else:
            df.loc[i,'studio_Fathom'] = 0
            
        if(df.loc[i,'studio'].find('Magn') != -1):
            df.loc[i,'studio_Magn'] = 1
        else:
            df.loc[i,'studio_Magn'] = 0
            
        if(df.loc[i,'studio'].find('IFC') != -1):
            df.loc[i,'studio_IFC'] = 1
        else:
            df.loc[i,'studio_IFC'] = 0
        
        if(df.loc[i,'studio'].find('WGUSA') != -1):
            df.loc[i,'studio_WGUSA'] = 1
        else:
            df.loc[i,'studio_WGUSA'] = 0
            
        if(df.loc[i,'studio'].find('KL') != -1):
            df.loc[i,'studio_KL'] = 1
        else:
            df.loc[i,'studio_KL'] = 0
    except:
        pass

In [4]:
# df[['name','studio','studio_uni', 'studio_WB', 'studio_Fox', 'studio_Sony', 'studio_STX',
#        'studio_LGF', 'studio_BV', 'studio_Focus']].head()

,name,studio,studio_uni,studio_WB,studio_Fox,studio_Sony,studio_STX,studio_LGF,studio_BV,studio_Focus
0,Black Panther,BV,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Avengers: Infinity War,BV,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Incredibles 2,BV,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Jurassic World: Fallen Kingdom,Uni.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aquaman,WB,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df[['name','rating']]

,name,rating
0,Black Panther,PG-13
1,Avengers: Infinity War,PG-13
2,Incredibles 2,PG
3,Jurassic World: Fallen Kingdom,PG-13
4,Aquaman,PG-13
5,Deadpool 2,R
6,Dr. Seuss' The Grinch (2018),PG
7,Mission: Impossible - Fallout,PG-13
8,Ant-Man and the Wasp,PG-13
9,Bohemian Rhapsody,PG-13


In [68]:
df['rating'].unique()

array(['PG-13', 'PG', 'R', 'Unrated', 'G', 'Unknown', 'Not Yet Rated',
       nan], dtype=object)

In [69]:
#rating
## cinemascore ##
#df['rating'].factorize()

df = pd.concat([df,pd.get_dummies(df['rating'],prefix='rating')],axis=1)
df = pd.concat([df,pd.get_dummies(df['cinemascore_rating'],prefix='CS_rating')],axis=1)

In [70]:
df.columns

Index(['#', 'name', 'distributor_full_name', 'release_date', 'genre',
       'runtime', 'rating', 'budget', 'rest_world', 'studio', 'total_gross',
       'theaters_total', 'opening', 'theaters_opening', 'open_date',
       'close_date', 'days_in_theaters', 'imdb_rating', 'directors',
       'cinemascore_name', 'cinemascore_rating', 'RT_name', 'RT_rating',
       'is_Action', 'is_Animation', 'is_Drama', 'is_Horror', 'is_Comedy',
       'is_Adventure', 'is_Music', 'is_Sci-Fi', 'is_Thriller',
       'is_Documentary', 'is_Romance', 'is_Fantasy', 'studio_uni', 'studio_WB',
       'studio_Fox', 'studio_Sony', 'studio_STX', 'studio_LGF', 'studio_BV',
       'studio_Focus', 'studio_Fathom', 'studio_Magn', 'studio_IFC',
       'studio_WGUSA', 'studio_KL', 'rating_G', 'rating_Not Yet Rated',
       'rating_PG', 'rating_PG-13', 'rating_R', 'rating_Unknown',
       'rating_Unrated', 'CS_rating_A', 'CS_rating_A+', 'CS_rating_A-',
       'CS_rating_B', 'CS_rating_B+', 'CS_rating_B-', 'CS_rating_C',


In [71]:
df = df.drop(['#'],axis=1)

In [72]:
df.to_csv('movies_clean.csv',index=False)

In [7]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)
#!dir

# 2. model

In [2]:
df = pd.read_csv('movies_clean.csv')
df.head()

,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,...,CS_rating_A+,CS_rating_A-,CS_rating_B,CS_rating_B+,CS_rating_B-,CS_rating_C,CS_rating_C+,CS_rating_C-,CS_rating_D+,CS_rating_D-
0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140,PG-13,NaN,6.468536e+08,BV,...,1,0,0,0,0,0,0,0,0,0
1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156,PG-13,NaN,1.369544e+09,BV,...,0,0,0,0,0,0,0,0,0,0
2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118,PG,NaN,6.342236e+08,BV,...,1,0,0,0,0,0,0,0,0,0
3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128,PG-13,170.0,8.917647e+08,Uni.,...,0,1,0,0,0,0,0,0,0,0
4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139,PG-13,NaN,8.127000e+08,WB,...,0,1,0,0,0,0,0,0,0,0


In [59]:
# del df['budget']
# df.columns

Index(['#', 'name', 'distributor_full_name', 'release_date', 'genre',
       'runtime', 'rating', 'rest_world', 'studio', 'total_gross',
       'theaters_total', 'opening', 'theaters_opening', 'open_date',
       'close_date', 'days_in_theaters', 'imdb_rating', 'directors',
       'cinemascore_rating', 'cinemascore_name'],
      dtype='object')

In [73]:
#brakujace wartosci - wykres
#df.isnull().sum().to_frame(name='count').sort_values(by='count', ascending=False)['count'].plot(kind='bar', figsize=(20,10))

dfna=df.dropna(axis=0)
len(dfna) #55

48

In [129]:
# feats = ['opening']
# X = df[ feats ].values
# y = df['total_gross'].values

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3) 

# X_train.shape, X_test.shape
#
#df['opening'].notnull().sum() <------- 2
#df['opening'].fillna(df['opening'].mean(),inplace=True) # nie powinieniem tak robic wiem ale to tylko do testu, bo dla dwoch wiereszy jest NA
#

target = 'total_gross'
#features = [c for c in dfna.columns if c != 'total_gross']
features = ['runtime', 'rest_world','theaters_total', 'opening', 'theaters_opening','days_in_theaters', 'imdb_rating']

train,test = train_test_split(dfna, test_size=0.33)

model = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
model.fit(train[features],train[target])
print(model.score(test[features],test[target]))

0.8936429740635896
